<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/ViT_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[github responsive ](https://github.com/jeonsworld/ViT-pytorch/blob/main/README.md)

In [1]:
!git clone https://github.com/jeonsworld/ViT-pytorch -q

In [2]:
!wget https://storage.googleapis.com/vit_models/imagenet21k/ViT-B_16.npz -q

In [7]:
!wget https://storage.googleapis.com/vit_models/imagenet21k/ViT-B_32.npz /checkpoint

--2024-04-13 13:59:51--  https://storage.googleapis.com/vit_models/imagenet21k/ViT-B_32.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.250.4.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419441810 (400M) [application/octet-stream]
Saving to: ‘ViT-B_32.npz’

ViT-B_32.npz        100%[===================>] 400.01M  12.1MB/s    in 36s     

2024-04-13 14:00:27 (11.2 MB/s) - ‘ViT-B_32.npz’ saved [419441810/419441810]

/content/ViT-pytorch/checkpoint: Scheme missing.
FINISHED --2024-04-13 14:00:27--
Total wall clock time: 36s
Downloaded: 1 files, 400M in 36s (11.2 MB/s)


In [ ]:
!unzip /content/drive/MyDrive/DATASETS/Common/real_gen_dataset.zip -d /content/ViT-pytorch/mydata

In [14]:
!unzip /content/drive/MyDrive/DATASETS/Common/RealFakeDB_tiny_test.zip -d /content/RealFakeDB_tiny_test

Archive:  /content/drive/MyDrive/DATASETS/Common/RealFakeDB_tiny_test.zip
   creating: /content/RealFakeDB_tiny_test/test/fake/
  inflating: /content/RealFakeDB_tiny_test/test/fake/000361dd-068d-4537-91f7-dcd63edf2d97.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/0044aeb7-0ec4-403b-9d52-2c8bf8503e93.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/006f7978-20d3-42f7-9c46-fb2692b5f551.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/00a16d0d-5c12-4edc-969e-18e8b6c64fc9.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/00b515c6-da36-4151-a232-b4ff42f20baf.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/00cd91a9-ce3d-4d89-990d-6cca3c65fdf5.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/012d762b-3c50-4d7f-99e0-740c68528d87.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/015b8967-ce45-40b6-9aa8-efac8537fa64.webp  
  inflating: /content/RealFakeDB_tiny_test/test/fake/017fe6aa-cbd1-4828-b284-d02a8069ac8e.webp  

In [3]:
%cd /content/ViT-pytorch

/content/ViT-pytorch


In [4]:
!pip install -r /content/ViT-pytorch/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Edit file train.py


```
#from apex import amp
import torch.cuda.amp as amp
from torch.cuda.amp import autocast, GradScaler

#from apex.parallel import DistributedDataParallel as DDP
from torch.nn.parallel import DistributedDataParallel as DDP

```



## Edit data_utils.py


```
import logging

import torch

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, SequentialSampler


logger = logging.getLogger(__name__)


def get_loader(args):
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()

    transform_train = transforms.Compose([
        transforms.RandomResizedCrop((args.img_size, args.img_size), scale=(0.05, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((args.img_size, args.img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    if args.dataset == "cifar10x":
        trainset = datasets.CIFAR10(root="./data",
                                    train=True,
                                    download=True,
                                    transform=transform_train)
        testset = datasets.CIFAR10(root="./data",
                                   train=False,
                                   download=True,
                                   transform=transform_test) if args.local_rank in [-1, 0] else None

    elif args.dataset=='chifar100':
        trainset = datasets.CIFAR100(root="./data",
                                     train=True,
                                     download=True,
                                     transform=transform_train)
        testset = datasets.CIFAR100(root="./data",
                                    train=False,
                                    download=True,
                                    transform=transform_test) if args.local_rank in [-1, 0] else None
    
    elif args.dataset == "cifar10":
        trainset = datasets.ImageFolder(root="./mydata",
                                    transform=transform_train) if args.local_rank in [-1, 0] else None
        testset = datasets.ImageFolder(root="./mydata",
                                    transform=transform_train) if args.local_rank in [-1, 0] else None
             
    else:
        return None
    if args.local_rank == 0:
        torch.distributed.barrier()

    train_sampler = RandomSampler(trainset) if args.local_rank == -1 else DistributedSampler(trainset)
    test_sampler = SequentialSampler(testset)
    train_loader = DataLoader(trainset,
                              sampler=train_sampler,
                              batch_size=args.train_batch_size,
                              num_workers=4,
                              pin_memory=True)
    test_loader = DataLoader(testset,
                             sampler=test_sampler,
                             batch_size=args.eval_batch_size,
                             num_workers=4,
                             pin_memory=True) if testset is not None else None

    return train_loader, test_loader

```



In [ ]:
!python train.py --name cifar10-100_500 --dataset cifar10 --model_type ViT-B_32 \
--pretrained_dir checkpoint/ViT-B_32.npz \
--train_batch_size 16 \
--eval_batch_size 16 \
--learning_rate 2e-4